![](./images/Apache_Hive_logo.svg)

# Hive

The Apache Hive ™ is a distributed, fault-tolerant data warehouse system that enables analytics at a massive scale and facilitates reading, writing, and managing petabytes of data residing in distributed storage using SQL.

## Hive Architecture

![Arch](./images/system_architecture.png)

* **UI** – The user interface for users to submit queries and other operations to the system. As of 2011 the system had a command line interface and a web based GUI was being developed.
* **Driver** – The component which receives the queries. This component implements the notion of session handles and provides execute and fetch APIs modeled on JDBC/ODBC interfaces.
* **Compiler** – The component that parses the query, does semantic analysis on the different query blocks and query expressions and eventually generates an execution plan with the help of the table and partition metadata looked up from the metastore.
* **Metastore** – The component that stores all the structure information of the various tables and partitions in the warehouse including column and column type information, the serializers and deserializers necessary to read and write data and the corresponding HDFS files where the data is stored.
* **Execution Engine** – The component which executes the execution plan created by the compiler. The plan is a DAG of stages. The execution engine manages the dependencies between these different stages of the plan and executes these stages on the appropriate system components.

## Hive Data Model

* **Tables** – These are analogous to Tables in Relational Databases. Tables can be filtered, projected, joined and unioned. Additionally all the data of a table is stored in a directory in HDFS. Hive also supports the notion of external tables wherein a table can be created on prexisting files or directories in HDFS by providing the appropriate location to the table creation DDL. The rows in a table are organized into typed columns similar to Relational Databases.
* **Partitions** – Each Table can have one or more partition keys which determine how the data is stored, for example a table T with a date partition column ds had files with data for a particular date stored in the <table location>/ds=<date> directory in HDFS. Partitions allow the system to prune data to be inspected based on query predicates, for example a query that is interested in rows from T that satisfy the predicate T.ds = '2008-09-01' would only have to look at files in <table location>/ds=2008-09-01/ directory in HDFS.
* **Buckets** – Data in each partition may in turn be divided into Buckets based on the hash of a column in the table. Each bucket is stored as a file in the partition directory. Bucketing allows the system to efficiently evaluate queries that depend on a sample of data (these are queries that use the SAMPLE clause on the table).*

## Internal vs External tables

* **Internal or Managed** tables are the default
    * Stored in HDFS directory 
    * DROP TABLE will remove the metadata and the HDFS data
* **External** tables are only metadata to external data in HDFS
    * DROP TABLE will only delete the metadata and leave the actual HDFS data

# Installation

## Build or Pull

* Build image (if needed)
    * `docker build -t hive .`
* Run the built image
    * `$ docker run -d -p 10000:10000 -p 10002:10002 -v $(pwd)/data:/data --env SERVICE_NAME=hiveserver2 --name hive -h hive hive:4.0.0`
* Alternatively run it from the official repo
    * `$ docker run -d -p 10000:10000 -p 10002:10002 -v $(pwd)/data:/data --env SERVICE_NAME=hiveserver2 --name hive -h hive apache/hive:4.0.0`

## Examine the installation

From inside the container:  
* `$ cd $HIVE_HOME/bin`  
* `$ beeline -u jdbc:hive2://localhost:10000`
* `0: jdbc:hive2://localhost:10000> show databases;`
* `0: jdbc:hive2://localhost:10000> use default;`
* `0: jdbc:hive2://localhost:10000> show tables;`

## Example

**Create external table**  
* Examine the sample csv file
    * Attach another shell
    * browse /data
    * `$ head 2010-summary.csv`
* Copy the file to HDFS
    * `$ hdfs dfs -mkdir -p /tmp/flight_data`
    * `$ hdfs dfs -ls /tmp`
    * `$ hdfs dfs -copyFromLocal 2010-summary.csv /tmp/flight_data`
* Create a database
    * `0: jdbc:hive2://localhost:10000> create database flight_data;`
* Create external table
    * `0: jdbc:hive2://localhost:10000> create external table fd2010_external(SourceCountry string, DestinationCountry string, count int) ROW FORMAT DELIMITED FIELDS TERMINATED BY ',' LOCATION '/tmp/flight_data';`
* Verify table content
    * `select * from fd2010_external limit 10;`

**Create internal table**  
* Create table
    * `create table fd2010_internal(SourceCountry string, DestinationCountry string, count int);`
    * `insert into table fd2010_internal select * from fd2010_external;`
* Examine the HDFS structure



**Example using ORC**  
* Create external table
    * `CREATE EXTERNAL TABLE IF NOT EXISTS trips (trip_id BIGINT, duration INT, start_date STRING, start_station STRING, start_terminal INT, end_date STRING, end_station STRING,  end_terminal INT, bike_number STRING, subscriber_type STRING, zip_code STRING)  STORED AS ORC LOCATION '/tmp/trips/';`
* Create Internal table (to enable ACID)
    * `CREATE TABLE IF NOT EXISTS inttrips (trip_id BIGINT, duration INT, start_date STRING, start_station STRING, start_terminal INT, end_date STRING, end_station STRING,  end_terminal INT, bike_number STRING, subscriber_type STRING, zip_code STRING) clustered by (trip_id) into 4 buckets  STORED AS ORC TBLPROPERTIES ('transactional'='true');`
* Load data from external into internal table
    * `insert into table inttrips select * from trips;`
* Test ACID (ex. Update)
    * `UPDATE inttrips SET duration = duration * 1.1 WHERE zip_code="95032";`

